In [3]:
import numpy as np
import math 
import functools
import random

N=24 #lateral size of MC lattice
J=[[-3.56E-3,1.06E-3,1.18E-3],[-2.29E-5,-3.56E-3,0.817],[-1.26E-3,5.20E-4,-3.56E-3]]
A11=1.75E-3
#A22=-6.25E-6
kB=1.38*pow(10,-23)
#Hstep=500
#step=5000
Hstep=50
step=1000



#local energy at site i
def local(s,NN):
    ea=A11*pow(s[2],2)
    ej=0
    for i in range(6):
        ej = ej+np.dot(s,np.dot(J,np.array(NN[i]).T))*0.5
    return ej
    
#to calculate the Energy of a specific spin structure spin[N][N][3]
def Energy(spin):
    E=0
    for b in range(N):
        for a in range(N):
            # tri lattice nearest neighbours:(a+-1,b),(a,b+-1),(a+1,b+1),(a-1,b-1)
            bl = b-1
            if bl < 0:
                bl = N-1
            bh = b + 1
            if bh>N-1:
                bh = 0
            al = a -1
            if al < 0:
                al = N-1
            ah= a +1
            if ah > N-1:
                ah = 0
            
            NN=[spin[bl][a],spin[bh][a],spin[b][al],spin[b][ah],spin[bl][al],spin[bh][ah]]
            E=E+local(spin[b][a],NN)
    return E
#delta=E'-E, if this step is rejected then return 0, if accepted return 1

def MCacceptance(delta,beta): 
    if delta < 0:
        return 1
    else:
        r = np.random.rand()
        if r <= math.exp(-1*delta*beta):
            return 1
        else:
            return 0

def randomspin():
    theta=np.random.rand()*np.pi
    phi=np.random.rand()*2*np.pi
    s=[np.math.sin(theta)*np.math.cos(phi),np.math.sin(theta)*np.math.sin(phi),np.math.cos(theta)]
    return s
        
def main():
    T=70 #temperature = 59.5K,transition temperature TN2
    beta = 1/(kB*T)
    spin=np.full((N,N,3),1) #FM initial state
    #spin=np.random.uniform(0,1,(5,5))
    f=open("mcsumS.txt","a")
    for t in range(10):
        #time
        time=0.2+t*0.01
        #heating
        for i in range(Hstep):
            #new structure at site(b,a)
            b=np.random.randint(0,N-1)
            a=np.random.randint(0,N-1)
        
            s=randomspin()
            bl = b - 1
            if bl < 0:
                bl = N-1
            bh = b + 1
            if bh>N-1:
                bh = 0
            al = a - 1
            if al < 0:
                al = N-1
            ah= a + 1
            if ah > N-1:
                ah = 0
            
            NN=[spin[bl][a],spin[bh][a],spin[b][al],spin[b][ah],spin[bl][al],spin[bh][ah]]
        
            #check 
            new=local(s,NN)
            old=local(spin[b][a],NN)
        
            if MCacceptance(new-old,beta):
                spin[b][a]=s
        
        
        #calculating
        ave=0
        avr_M=0
        avr_Msquare=0
        
        for i in range(step):
            #new structure at site(b,a)
            b=np.random.randint(0,N-1)
            a=np.random.randint(0,N-1)
        
            s=randomspin()
            bl = b - 1
            if bl < 0:
                bl = N-1
            bh = b + 1
            if bh>N-1:
                bh = 0
            al = a - 1
            if al < 0:
                al = N-1
            ah= a + 1
            if ah > N-1:
                ah = 0
            
            NN=[spin[bl][a],spin[bh][a],spin[b][al],spin[b][ah],spin[bl][al],spin[bh][ah]]
        
            #check 
            new=local(s,NN)
            old=local(spin[b][a],NN)
        
            if MCacceptance(new-old,beta):
                spin[b][a]=s
                
            #per atom
            
            E=Energy(spin)/(N*N)
            ave=ave+E
            
            
            M=[0,0,0]
            Msquare=0
            for j in range(N):
                for k in range(N):
                    M=M+spin[j][k]
                    Msquare=Msquare+pow(spin[j][k][0],2)+pow(spin[j][k][1],2)+pow(spin[j][k][2],2)
            M=M/(N*N)
            Msquare=Msquare/(N*N)
            
            avr_M=avr_M+M
            avr_Msquare=avr_Msquare+Msquare
        
        ave=ave/step
        avr_M=avr_M/step
        avr_Msquare=avr_Msquare/step
        f.write(str(time)+" "+str(ave)+" "+str(avr_M)+" "+str(avr_Msquare)+"\n")
        #print("time=",time,"average Energy per sit=",ave,"average magnetic moment",avr_M,"average module of magnetic moment",np.sqrt(avr_Msquare))
    
    f.close()

if __name__=='__main__':
    main()
            
            
        
    
    
    
    